In [89]:
# importing the required libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd 
import re

print('Import Done')

Import Done


<h3> Scrapping the Data from the Wikepdia page using the BeatifulSoup Library </h3>

In [122]:
#Given URL of the Wikepedia Page
URL='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page =  requests.get(URL)

soup = BeautifulSoup(page.content, 'html.parser')

#find the table data in the returned dataset from the BeautifulSoup Library
postal_code_data = soup.find('table', attrs={'class':'wikitable sortable'})

#Defining the Regex expression to remove HTML tags 
cleanr = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')

#finding all the values within the <td>..</td> elements
postal_code_data = postal_code_data.tbody.find_all('td')


In [183]:
#empty lists to store the table data
table_data=[]
temp_list=[]

#iterating through the reruned data from Beautifulsoup to extract the values within the <td>..</td> elements
for item in postal_code_data:
    
    # Since we know each row contains only 3 columns, hence appennding 3 items at once to a temp list to construct a row. 
    if len(temp_list) < 3: 
        temp_list.append(re.sub(cleanr, '', str(item).replace('\n', '').strip()))
    
    #Once the temp list is populated with 3 items i.e. 1 row, we empty the list, append the temp list to the table data list
    #and continue same as above.
    
    else:
        table_data.append(temp_list)
        temp_list=[]
        temp_list.append(re.sub(cleanr, '', str(item).replace('\n', '').strip()))
       
# Appending the last row to the table_data
table_data.append(temp_list)


<h3> Data Preparation </h3>

In [199]:
# Creating a DataFrame for postal codes of Canada
post_code_df = pd.DataFrame(table_data, columns=['PostalCode', 'Borough', 'Neighbourhood'])

# Removing rows with not assigned Boroughs
post_code_df = post_code_df[post_code_df['Borough']!='Not assigned']

# Replacing Not assigned neigbourhoods with the borough's names 
post_code_df['Neighbourhood'] = post_code_df.apply(lambda x : x.Borough  if x.Neighbourhood=='Not assigned' else x.Neighbourhood, axis=1 )
post_code_df

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [201]:
post_code_df.shape

(103, 3)